In [ ]:
pip install requests

In [3]:
import requests
import pandas as pd

# Define the base URL for the BC Data Catalogue API
BASE_URL = "https://catalogue.data.gov.bc.ca/api/3/action/"

def get_dataset_info(dataset_id):
    """
    Fetches dataset information for a given dataset ID from the BC Data Catalogue API.
    
    :param dataset_id: The ID of the dataset to retrieve information for.
    :return: A dictionary containing the dataset information.
    """
    url = f"{BASE_URL}package_show"
    params = {"id": dataset_id}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
        
        if data['success']:
            return data['result']
        else:
            print("Error in API response")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

def fetch_resource_data(resource_url, resource_format):
    """
    Fetches and loads data from a resource URL.
    
    :param resource_url: The URL of the resource to fetch data from.
    :param resource_format: The format of the resource (e.g., 'CSV').
    :return: The data loaded into a Pandas DataFrame or appropriate structure.
    """
    try:
        if resource_format.lower() == 'csv':
            return pd.read_csv(resource_url)
        elif resource_format.lower() == 'json':
            return pd.read_json(resource_url)
        elif resource_format.lower() == 'xlsx':
            return pd.read_excel(resource_url)
        else:
            print(f"Unsupported format: {resource_format}")
            return None
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None

def main():
    # Example dataset ID for BC Headset data
    dataset_id = "labour-market-outlook" #"f9566991-eb97-49a9-a587-5f0725024985"
    
    # Fetch dataset information
    dataset_info = get_dataset_info(dataset_id)
    
    if dataset_info:
        print("Dataset Information:")
        print(f"Title: {dataset_info['title']}")
        print(f"Metadata Created: {dataset_info['metadata_created']}")
        print(f"Metadata Modified: {dataset_info['metadata_modified']}")
        print(f"Author: {dataset_info['author']}")
        print(f"Notes: {dataset_info['notes']}")
        
        # Print out the resources available in this dataset
        print("\nResources:")
        for resource in dataset_info['resources']:
            print(f"- Name: {resource['name']}")
            print(f"  URL: {resource['url']}")
            print(f"  Format: {resource['format']}")
            print()
        
        # Specify the resource name you want to load
        target_resource_name = "Job Openings by NOC and Skill Cluster"
        
        # Find the specified resource
        for resource in dataset_info['resources']:
            if resource['name'] == target_resource_name:
                print(f"\nLoading data from selected resource: {resource['name']}")
                data = fetch_resource_data(resource['url'], resource['format'])
                
                if data is not None:
                    print(f"Data from resource '{resource['name']}':")
                    print(data.head())  # Display the first few rows of the data
                    return data
                else:
                    print("Failed to load data.")
                break
        else:
            print(f"Resource '{target_resource_name}' not found.")

if __name__ == "__main__":
    main()

Dataset Information:
Title: Labour Market Outlook
Metadata Created: 2015-01-28T22:20:58.620619
Metadata Modified: 2023-11-28T18:08:30.332116
Author: 55f3553e-9b91-41fa-b0b4-a546fad16f9f
Notes: Each year a report forecasting BC's labour market needs over the coming decade is produced by the BC government. The report looks at employment supply and demand by occupation and industry for each of the province's regions. 

When utilizing this data, please cite as follows:
Labour Market Outlook, Labour Market Information Office, Ministry of Post-Secondary Education and Future Skills, Government of British Columbia.

Resources:
- Name: Employment by Ind and Occ for BC and Regions
  URL: https://catalogue.data.gov.bc.ca/dataset/f9566991-eb97-49a9-a587-5f0725024985/resource/aa195dc1-f3e8-413a-b38c-24af95a3276e/download/employment-by-ind-and-occ-for-bc-and-regions.xlsx
  Format: csv

- Name: Employment by Industry and Occupation for BC
  URL: https://catalogue.data.gov.bc.ca/dataset/f9566991-eb97-